# Определение токсичности коментариев

Задача на классификацию: создать модель, определяющую токсичность коментариев (позитивные/негативные коментарии). 

Текст коментариев - английский.

Оценка работы модели - **'F1'** не ниже 0.45

## Подготовка

### Подключим модули

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download(['averaged_perceptron_tagger'])
from tqdm import notebook, tqdm
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Загрузим данные и посмотрим на них

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')
print(df.head())
df.info()

                                                text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Данные содержат много лишних символов. Пропусков нет.

### Очистка и лемматизация

In [4]:
%%time
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = text.split()
    text = ' '.join(text)
    return text

def get_wordnet_pos(text):
    tag = nltk.pos_tag([text])[0][1][0].upper()
    tag_dic = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV        
    }
    return tag_dic.get(tag, wordnet.NOUN)

def lemmatize(text):
    tokens=nltk.word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(t, pos=get_wordnet_pos(t)) for t in tokens]
    lem = ' '.join(lem)
    return lem

tqdm.pandas()
df['corpus'] = df['text'].apply(clear_text).progress_apply(lemmatize)

# Wall time: 37min 40s

100%|██████████| 159571/159571 [34:13<00:00, 77.70it/s] 

CPU times: user 30min 43s, sys: 2min 42s, total: 33min 26s
Wall time: 34min 16s


Сохранимся

In [5]:
df.to_csv('df_with_corpus.csv')

Загрузимся

In [17]:
df = pd.read_csv('df_with_corpus.csv', index_col = [0])

Проверим на пропуски

In [18]:
df[df.corpus != df.corpus]

,text,toxic,corpus
4482,1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n...,0,NaN
6300,193.61.111.53 15:00,0,NaN
17311,~ \n\n68.193.147.157,0,NaN
52442,"14:53,",0,NaN
53787,92.24.199.233|92.24.199.233]],0,NaN
61758,"""\n\n 199.209.144.211 """,0,NaN
82681,"""\n '''''' 2010/2013 """,0,NaN


Видимо пропуски появились после очистки, уберём их

In [19]:
df = df.dropna(axis=0)

### Разделим выборки (trani/test = 80/20)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(df.corpus, 
                                                    df.toxic, 
                                                    test_size=0.2,
                                                    random_state=12345)

print(x_train.shape[0]/len(df), x_test.shape[0]/len(df))

0.7999987465844426 0.20000125341555738


### Создадим мешок слов с помощью tf-idf

В качестве параметров передадим словарь `english`, содержащий стоп-слова

In [21]:
%%time

stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

x_train = count_tf_idf.fit_transform(x_train)

x_test = count_tf_idf.transform(x_test)

CPU times: user 10.1 s, sys: 24.3 ms, total: 10.1 s
Wall time: 10.3 s


Проверим размеры выборок

In [22]:
print(x_train.shape[0]/len(df), x_test.shape[0]/len(df))

print(len(df) == x_train.shape[0] + x_test.shape[0])

0.7999987465844426 0.20000125341555738
True


### Проверим баланс классов

In [23]:
print(df.toxic.value_counts())

# и долю классов 1 к 0
print(16225/len(df))

0    143339
1     16225
Name: toxic, dtype: int64
0.10168333709358


Дисбаланс классов существеннен, учтём это когда будем обучать модели

## Обучение

### Создадим scorer для GridSearchCV, для подбора параметров моделей (бинарная классификация)

In [24]:
f1 = make_scorer(f1_score , average='binary')

### DummyClassifier

In [25]:
model = DummyClassifier(strategy='constant', constant=1, random_state=12345).fit(x_train, y_train)
predict = model.predict(x_test)
print('F1:', f1_score(y_test, predict, average='binary'))

F1: 0.18092170889503237


### LogisticRegression

Через GridSearchCV подберём модели параметры

In [ ]:
# без pipeline

# %%time

# model = LogisticRegression(
#     random_state=12345, 
#     class_weight='balanced', 
#     solver='liblinear')

# params = {
#     "C": [0.01, 0.1, 1, 10, 100], 
#     "penalty":["l1", "l2"],
# }

# grid = GridSearchCV(model, params, cv=5, scoring=f1, verbose=3)

# grid.fit(x_train, y_train)

# print('Лучшие параметры ', grid.best_params_)
# print('Лучший результат ', grid.best_score_)

# model = LogisticRegression(random_state=12345, **grid.best_params_)
# model.fit(x_train, y_train)
# predict = model.predict(x_test)

# print('f1 на тестовой выборке = ', f1_score(y_test, predict, average='binary'))

# Лучшие параметры  {'C': 10, 'penalty': 'l2'}
# Лучший результат  0.7599877694299754
# f1 на тестовой выборке =  0.774667599720084
# Wall time: 12min 10s

In [29]:
# с pipeline

stopwords = set(nltk_stopwords.words('english'))

x_train, x_test, y_train, y_test = train_test_split(df.corpus, 
                                                    df.toxic, 
                                                    test_size=0.2,
                                                    random_state=12345)

params = {
    'classifier__C': [0.01, 0.1, 1, 10, 100], 
    'classifier__penalty': ["l1", "l2"], 
    'classifier__random_state': [12345], 
    'classifier__class_weight': ['balanced'],
    'classifier__solver': ['liblinear'], 
}

steps = [('tfidf', TfidfVectorizer(stop_words=stopwords)), 
         ('classifier', LogisticRegression())]

pipe = Pipeline(steps)

grid = GridSearchCV(pipe, params, cv=5, scoring=f1, verbose=0)

grid.fit(x_train, y_train)

print('Лучшие параметры ', grid.best_params_)
print('Лучший результат ', grid.best_score_)

#  Лучшие параметры  {'classifier__C': 10, 
#                     'classifier__class_weight': 'balanced', 
#                     'classifier__penalty': 'l2', 
#                     'classifier__random_state': 12345, 
#                     'classifier__solver': 'liblinear'}

# Лучший результат  0.7598857119294717

Лучшие параметры  {'classifier__C': 10, 'classifier__class_weight': 'balanced', 'classifier__penalty': 'l2', 'classifier__random_state': 12345, 'classifier__solver': 'liblinear'}
Лучший результат  0.7598857119294717


In [30]:
x_train, x_test, y_train, y_test = train_test_split(df.corpus, 
                                                    df.toxic, 
                                                    test_size=0.2,
                                                    random_state=12345)

stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

x_train = count_tf_idf.fit_transform(x_train)

x_test = count_tf_idf.transform(x_test)


parameters = {}

for key in grid.best_params_.keys():
    parameters[key[12:]] = grid.best_params_[key]

model = LogisticRegression(**parameters)

model.fit(x_train, y_train)

predict = model.predict(x_test)

print('f1 на тестовой выборке = ', f1_score(y_test, predict, average='binary'))

# f1 на тестовой выборке =  0.7556521739130434

f1 на тестовой выборке =  0.7556521739130434


### RandomForestClassifier

In [32]:
%%time

model = RandomForestClassifier(random_state=12345, class_weight='balanced')

params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 7]
}

grid = GridSearchCV(model, params, cv=5, scoring=f1, verbose=0)

grid.fit(x_train, y_train)

print('Лучшие параметры ', grid.best_params_)
print('Лучший результат ', grid.best_score_)

model = RandomForestClassifier(random_state=12345, **grid.best_params_)
model.fit(x_train, y_train)
predict = model.predict(x_test)

print('f1 на тестовой выборке = ', f1_score(y_test, predict, average='binary'))

# Лучшие параметры  {'max_depth': 7, 'n_estimators': 100}
# Лучший результат  0.33645423766078303
# f1 на тестовой выборке =  0.0
# Wall time: 5min 47s

Лучшие параметры  {'max_depth': 7, 'n_estimators': 100}
Лучший результат  0.33645423766078303
f1 на тестовой выборке =  0.0
CPU times: user 5min 26s, sys: 0 ns, total: 5min 26s
Wall time: 5min 31s


### CatBoostClassifier

In [34]:
model = CatBoostClassifier(eval_metric='F1', verbose=3, random_state=12345)
model.fit(x_train, y_train)
predict = model.predict(x_test)

print('f1 на тестовой выборке = ', f1_score(y_test, predict, average='binary'))

# f1 на тестовой выборке =  0.7511111111111112 > 0.75 

Learning rate set to 0.081697
0:	learn: 0.4733322	total: 4.46s	remaining: 1h 14m 15s
3:	learn: 0.4750345	total: 16.6s	remaining: 1h 9m
6:	learn: 0.4635362	total: 27.8s	remaining: 1h 5m 41s
9:	learn: 0.4902888	total: 38.7s	remaining: 1h 3m 48s
12:	learn: 0.4896607	total: 50.1s	remaining: 1h 3m 21s
15:	learn: 0.4916163	total: 1m 4s	remaining: 1h 6m 24s
18:	learn: 0.5147183	total: 1m 15s	remaining: 1h 5m 8s
21:	learn: 0.5037832	total: 1m 26s	remaining: 1h 4m 14s
24:	learn: 0.5213899	total: 1m 41s	remaining: 1h 5m 41s
27:	learn: 0.5255312	total: 1m 51s	remaining: 1h 4m 42s
30:	learn: 0.5275262	total: 2m 3s	remaining: 1h 4m 35s
33:	learn: 0.5298131	total: 2m 14s	remaining: 1h 3m 53s
36:	learn: 0.5353870	total: 2m 25s	remaining: 1h 3m
39:	learn: 0.5391946	total: 2m 35s	remaining: 1h 2m 17s
42:	learn: 0.5432980	total: 2m 46s	remaining: 1h 1m 35s
45:	learn: 0.5544479	total: 2m 56s	remaining: 1h 1m
48:	learn: 0.5659501	total: 3m 6s	remaining: 1h 20s
51:	learn: 0.5795588	total: 3m 16s	remaining:

## Выводы

* Из трёх рассматриваемых моделей лучший результат показала модель LogisticRegression. 
* Модель CatBoostClassifier заняла второе место (с учётом стандартных настроек)
* Модель RandomForesterClassifier не смогла обучиться на данных.
